# Session setup

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [3]:
!cat /proc/meminfo

MemTotal:       13333560 kB
MemFree:        10616348 kB
MemAvailable:   12452808 kB
Buffers:          102356 kB
Cached:          1884932 kB
SwapCached:            0 kB
Active:           996948 kB
Inactive:        1397332 kB
Active(anon):     382732 kB
Inactive(anon):      404 kB
Active(file):     614216 kB
Inactive(file):  1396928 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               460 kB
Writeback:             0 kB
AnonPages:        406936 kB
Mapped:           216004 kB
Shmem:              1008 kB
Slab:             177656 kB
SReclaimable:     132172 kB
SUnreclaim:        45484 kB
KernelStack:        4304 kB
PageTables:         5612 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666780 kB
Committed_AS:    3026328 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1040 kB
AnonHugePages:   

## Mount drive

In [3]:
# Mount the google drive
from google.colab import drive
import os
import pandas as pd

### GLOBALS ###
BASE_PATH = ''
DATA_PATH = ''
MODEL_PATH = ''

def mount_drive():
    '''Mounts google drive and sets base/data paths'''
    drive.mount('/content/drive')

    global BASE_PATH
    global DATA_PATH
    global MODEL_PATH

    # Set up file paths
    BASE_PATH = '/content/drive/MyDrive/bert/'
    DATA_PATH = os.path.join(BASE_PATH, 'data')
    MODEL_PATH = os.path.join(BASE_PATH, 'models')

# Mount the drive
mount_drive()

print(f"BASE_PATH = {BASE_PATH} exists: {os.path.isdir(BASE_PATH)}")
print(f"DATA_PATH = {DATA_PATH} exists: {os.path.isdir(DATA_PATH)}")
print(f"MODEL_PATH = {MODEL_PATH} exists: {os.path.isdir(MODEL_PATH)}")

Mounted at /content/drive
BASE_PATH = /content/drive/MyDrive/bert/ exists: True
DATA_PATH = /content/drive/MyDrive/bert/data exists: True
MODEL_PATH = /content/drive/MyDrive/bert/models exists: True


## Install transformers

In [6]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ws48rogn
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ws48rogn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 5.4MB/s 
     |████████████████████████████████| 901kB 31.4MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2168813 sha256=d9e8d7ea078aca1978060cadee945f86b4f17eb7f2961ded9ea8ffd1bd5f0f04
  Stored in directory: /tmp/pip-ephem-wheel-cache-n7k_829_/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers


## Check gpu

In [5]:
!nvidia-smi

Mon May  3 14:59:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Model setup

## Instantiate Tokenizer

In [7]:
from transformers import RobertaTokenizerFast
fast_tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

## Set up config

In [8]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

## Get model architecture



In [9]:
tmp_model = RobertaForMaskedLM(config=config)
print(tmp_model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

# MLM

## Set up trainers

In [8]:
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pickle

def train_mlm(model, filepath, save_path='tmp_model', epochs=1):
    dataset = LineByLineTextDataset(
        tokenizer=fast_tokenizer,
        file_path=filepath,
        block_size=128,
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=fast_tokenizer, 
        mlm=True, 
        mlm_probability=0.15
    )

    current_model_path = os.path.join(MODEL_PATH, 'tmp')
    # Define training
    training_args = TrainingArguments(
        output_dir=save_path,
        overwrite_output_dir=True,
        num_train_epochs=epochs,
        per_device_train_batch_size=64,
        save_steps=10_000,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    loss = trainer.train()

    # Save full model
    model_save_path = os.path.join(save_path, 'full_model')
    trainer.save_model(model_save_path)

    # Save base model
    base_model_save_path = os.path.join(save_path, 'base_model')
    model.base_model.save_pretrained(base_model_save_path)

    return loss


def training_run(texts: list, order: list, epochs: list, base_path: str, N: int=5):
    # Ensure subdirectories exist:
    for i in range(N):
        subdir = os.path.join(base_path, str(i))
        if not os.path.exists(subdir):
            print(f"Making dir: {subdir}")
            os.makedirs(subdir)

    # Instantiate model & trainer
    all_losses = []
    for i in range(N):
        full_save_path = os.path.join(base_path, str(i))
        model = RobertaForMaskedLM(config=config)
        running_losses = []

        for i in order:
            text, num_epochs = texts[i], epochs[i]
            print(f"Training on {text} for {num_epochs} epochs...")
            text_path = os.path.join(DATA_PATH, text)

            losses = train_mlm(model=model, 
                            filepath=text_path, 
                            save_path=full_save_path, 
                            epochs=num_epochs)
            running_losses += losses # concatenate to previous list of losses

            # Save the losses
            loss_filepath = os.path.join(full_save_path, 'losses.p')

        print("Finished training...")
        pickle.dump(running_losses, open(loss_filepath, 'wb'))
        all_losses.append(running_losses)

    return all_losses

## Set up texts

In [10]:
# Load file paths and compute # epochs
import math

texts = ['rageddyann.txt', 
         'secretgarden.txt', 
         'treasureisland.txt', 
         'reuters.txt',
         'taleoftwocities.txt',
         'wiki.txt',
         'analysisofmind.txt']

text_paths = [os.path.join(DATA_PATH, text) for text in texts]
sizes = [os.path.getsize(path) for path in text_paths]
largest = max(sizes)
epochs = [math.ceil(largest / (size*2)) for size in sizes]
rel_size_weights = [largest / size for size in sizes]
print(epochs)
print(sizes)

[56, 11, 13, 1, 7, 1, 9]
[88453, 455570, 379886, 8868430, 787404, 9822679, 549116]
[111.0496987100494, 21.561294641877208, 25.85691233685895, 1.1076006688895328, 12.474763907727164, 1.0, 17.88816752744411]


## Run various permutations

In [13]:
loss_dict = dict()

In [15]:
# # test permutation
# losses = dict()
# save_path = os.path.join(MODEL_PATH, 'test')
# ordering = [0, 1]
# loss_dict['test'] = training_run(texts=texts[:2],
#                               order=ordering,
#                               epochs=[1, 1],
#                               base_path=save_path,
#                               N=2)

Making dir: /content/drive/MyDrive/bert/models/test/0
Making dir: /content/drive/MyDrive/bert/models/test/1
Training on rageddyann.txt for 1 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss


Training on secretgarden.txt for 1 epochs...


Step,Training Loss


Finished training...
Training on rageddyann.txt for 1 epochs...


Step,Training Loss


Training on secretgarden.txt for 1 epochs...


Step,Training Loss


Finished training...


## 1 epoch per text

In [35]:
one_per = [1] * len(texts)
print(one_per)

[1, 1, 1, 1, 1, 1, 1]


In [38]:
#@title In order
save_path = os.path.join(MODEL_PATH, 'in_order_singlepass')
ordering = list(range(len(texts)))
loss_dict['in_order_singlepass'] = training_run(texts=texts, 
                                  order=ordering, 
                                  epochs=one_per, 
                                  base_path=save_path,
                                  N=1)

Training on rageddyann.txt for 1 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss


Training on secretgarden.txt for 1 epochs...


Step,Training Loss


...

Training on wiki.txt for 1 epochs...


Step,Training Loss
500,7.467200


Training on analysisofmind.txt for 1 epochs...


Step,Training Loss


Finished training...


In [39]:
#@title reverse order
save_path = os.path.join(MODEL_PATH, 'reverse_order_singlepass')
ordering = list(range(len(texts) - 1, -1, -1))
loss_dict['reverse_order_singlepass'] = training_run(texts=texts, 
                                  order=ordering, 
                                  epochs=one_per, 
                                  base_path=save_path,
                                  N=1)

Making dir: /content/drive/MyDrive/bert/models/reverse_order_singlepass/0
Training on analysisofmind.txt for 1 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss


Training on wiki.txt for 1 epochs...


Step,Training Loss
500,7.872600


...

Training on secretgarden.txt for 1 epochs...


Step,Training Loss


Training on rageddyann.txt for 1 epochs...


Step,Training Loss


Finished training...


In [40]:
#@title Random order
import numpy as np

rand_path_1 = os.path.join(MODEL_PATH, 'random_order_singlepass')
ordering = list(np.random.permutation(len(texts)))
print(f"Ordering: {ordering}")
loss_dict['random_order_singlepass'] = training_run(texts=texts,
                                 order=ordering,
                                 epochs=one_per,
                                 base_path=rand_path_1,
                                 N=1)

Ordering: [0, 1, 5, 2, 4, 3, 6]
Making dir: /content/drive/MyDrive/bert/models/random_order_singlepass/0
Training on rageddyann.txt for 1 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss


Training on secretgarden.txt for 1 epochs...


Step,Training Loss


...

Training on reuters.txt for 1 epochs...


Step,Training Loss
500,5.782600
1000,5.183900
1500,4.855000
2000,4.734600
2500,4.611500


Training on analysisofmind.txt for 1 epochs...


Step,Training Loss


Finished training...


## Two per

In [ ]:
two_per = [2] * len(texts)

In [1]:
size_kb = [88, 456, 380, 9114, 787, 10035, 529]
print(sum(size_kb))



21389


## Exponential

In [43]:
train_lens = [2**(len(texts) - i - 1) for i in range(len(texts))]
print(train_lens)

[64, 32, 16, 8, 4, 2, 1]


In [44]:
#@title In order
save_path = os.path.join(MODEL_PATH, 'in_order_exp')
ordering = list(range(len(texts)))
loss_dict['in_order_exp'] = training_run(texts=texts, 
                                  order=ordering, 
                                  epochs=train_lens, 
                                  base_path=save_path,
                                  N=1)

Making dir: /content/drive/MyDrive/bert/models/in_order_exp/0
Training on rageddyann.txt for 64 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss
500,6.372000
1000,4.996300
1500,4.548600


Training on secretgarden.txt for 32 epochs...


Step,Training Loss
500,5.266900
1000,4.302000
1500,3.902700
2000,3.647700
2500,3.498500
3000,3.388200
3500,3.294100


...

Training on wiki.txt for 2 epochs...


Step,Training Loss
500,6.801800
1000,6.397500
1500,6.239400


Training on analysisofmind.txt for 1 epochs...


Step,Training Loss


Finished training...


## Training time proportional to text

In [17]:
#@title In order
save_path = os.path.join(MODEL_PATH, 'in_order')
ordering = list(range(len(texts)))
loss_dict['in_order'] = training_run(texts=texts, 
                                  order=ordering, 
                                  epochs=epochs, 
                                  base_path=save_path)

Making dir: /content/drive/MyDrive/bert/models/in_order/0
Making dir: /content/drive/MyDrive/bert/models/in_order/1
Making dir: /content/drive/MyDrive/bert/models/in_order/2
Making dir: /content/drive/MyDrive/bert/models/in_order/3
Making dir: /content/drive/MyDrive/bert/models/in_order/4
Training on rageddyann.txt for 56 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss
500,6.358200
1000,5.053600


Training on secretgarden.txt for 11 epochs...


Step,Training Loss
500,5.419400
1000,4.546600


...

Training on wiki.txt for 1 epochs...


Step,Training Loss
500,7.225400


Training on analysisofmind.txt for 9 epochs...


Step,Training Loss
500,5.130800
1000,4.560500


Finished training...


In [24]:
#@title Reverse order
rev_path = os.path.join(MODEL_PATH, 'reverse_order')
ordering = list(range(len(texts) - 1, -1, -1))
loss_dict['reverse_order'] = training_run(texts=texts,
                                       order=ordering,
                                       epochs=epochs,
                                       base_path=rev_path,
                                       N=1)


Making dir: /content/drive/MyDrive/bert/models/reverse_order/0
Making dir: /content/drive/MyDrive/bert/models/reverse_order/1
Making dir: /content/drive/MyDrive/bert/models/reverse_order/2
Making dir: /content/drive/MyDrive/bert/models/reverse_order/3
Making dir: /content/drive/MyDrive/bert/models/reverse_order/4
Training on analysisofmind.txt for 9 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss
500,6.998900
1000,6.201300


Training on wiki.txt for 1 epochs...


Step,Training Loss
500,7.697100


...

Training on secretgarden.txt for 11 epochs...


Step,Training Loss
500,4.273200
1000,3.754100


Training on rageddyann.txt for 56 epochs...


Step,Training Loss
500,4.082300
1000,3.133600


Finished training...


In [26]:
#@title Random permutation 1
import numpy as np

rand_path_1 = os.path.join(MODEL_PATH, 'random_order_1')
ordering = list(np.random.permutation(len(texts)))
print(f"Ordering: {ordering}")
loss_dict['random1'] = training_run(texts=texts,
                                 order=ordering,
                                 epochs=epochs,
                                 base_path=rand_path_1,
                                 N=1)

Ordering: [0, 1, 5, 2, 4, 3, 6]
Making dir: /content/drive/MyDrive/bert/models/random_order_1/0
Training on rageddyann.txt for 56 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss
500,6.372000
1000,5.042300


Training on secretgarden.txt for 11 epochs...


Step,Training Loss
500,5.405600
1000,4.522500


...

Training on reuters.txt for 1 epochs...


Step,Training Loss
500,5.112700
1000,4.477700
1500,4.222000
2000,4.146600
2500,4.040800


Training on analysisofmind.txt for 9 epochs...


Step,Training Loss
500,5.158800
1000,4.571400


Finished training...


In [27]:
#@title Random permutation 2
rand_path_2 = os.path.join(MODEL_PATH, 'random_order_2')
ordering = list(np.random.permutation(len(texts)))
print(f"Ordering: {ordering}")
loss_dict['random2'] = training_run(texts=texts,
                                 order=ordering,
                                 epochs=epochs,
                                 base_path=rand_path_2,
                                 N=1)

Ordering: [0, 1, 5, 2, 4, 3, 6]
Making dir: /content/drive/MyDrive/bert/models/random_order_2/0
Training on rageddyann.txt for 56 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss
500,6.361600
1000,5.046400


Training on secretgarden.txt for 11 epochs...


Step,Training Loss
500,5.404300
1000,4.536500


...

Training on reuters.txt for 1 epochs...


Step,Training Loss
500,5.116000
1000,4.489400
1500,4.228600
2000,4.148700
2500,4.046100


Training on analysisofmind.txt for 9 epochs...


Step,Training Loss
500,5.167300
1000,4.577200


Finished training...


In [28]:
#@title Random permutation 3
rand_path_3 = os.path.join(MODEL_PATH, 'random_order_3')
ordering = list(np.random.permutation(len(texts)))
print(f"Ordering: {ordering}")
loss_dict['random3'] = training_run(texts=texts,
                                 order=ordering,
                                 epochs=epochs,
                                 base_path=rand_path_3,
                                 N=1)

Ordering: [0, 1, 5, 2, 4, 3, 6]
Making dir: /content/drive/MyDrive/bert/models/random_order_3/0
Training on rageddyann.txt for 56 epochs...


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Step,Training Loss
500,6.361600
1000,5.046400


Training on secretgarden.txt for 11 epochs...


Step,Training Loss
500,5.404300
1000,4.536500


...

Training on reuters.txt for 1 epochs...


Step,Training Loss
500,5.116000
1000,4.489400
1500,4.228600
2000,4.148700
2500,4.046100


Training on analysisofmind.txt for 9 epochs...


Step,Training Loss
500,5.167300
1000,4.577200


Finished training...


In [33]:
#@title Save results
save_path = os.path.join(DATA_PATH, 'loss_dict.p')
pickle.dump(loss_dict, open(save_path, 'wb'))

## Try forward and reverse with only 1 epoch per text

In [ ]:
epochs = [1] * len(texts)
forward_order = list(range(len(texts)))
save_path = os.path.join(MODEL_PATH, 'in_order_once')
loss_dict['in_order_once'] = training_run(texts=texts, 
                                  order=forward_order, 
                                  epochs=epochs, 
                                  base_path=save_path)

In [ ]:
rev_path = os.path.join(MODEL_PATH, 'reverse_order_once')
rev_ordering = list(range(len(texts) - 1, -1, -1))
loss_dict['reverse_order'] = training_run(texts=texts,
                                       order=rev_ordering,
                                       epochs=epochs,
                                       base_path=rev_path)

In [ ]:
rev_order = 

In [ ]:
# attach new head
from transformers import RobertaForSequenceClassification

# get the correct path for the base model
new_model = RobertaForSequenceClassification.from_pretrained('testdir')
print(new_model)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at testdir and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="tmp_model",
    tokenizer=fast_tokenizer
)

In [ ]:
fill_mask("All children, except <mask>, grow up.")

[{'score': 0.02480253390967846,
  'sequence': 'All children, except to, grow up.',
  'token': 7,
  'token_str': ' to'},
 {'score': 0.02457950823009014,
  'sequence': 'All children, except and, grow up.',
  'token': 8,
  'token_str': ' and'},
 {'score': 0.022292720153927803,
  'sequence': 'All children, except the, grow up.',
  'token': 5,
  'token_str': ' the'},
 {'score': 0.019803442060947418,
  'sequence': 'All children, except was, grow up.',
  'token': 21,
  'token_str': ' was'},
 {'score': 0.015464212745428085,
  'sequence': 'All children, except a, grow up.',
  'token': 10,
  'token_str': ' a'}]

In [ ]:
from transformers import RobertaForSequenceClassification

MODEL_PATH = os.path.join(BASE_PATH, 'models')
for i in range(5):
    print(os.path.join(MODEL_PATH, str(i)))
    # model = RobertaForMaskedLM(config=config)
    # model.base_model.save_pretrained('foo')
model2 = RobertaForSequenceClassification.from_pretrained('foo')

/content/drive/MyDrive/bert/models/0
/content/drive/MyDrive/bert/models/1
/content/drive/MyDrive/bert/models/2
/content/drive/MyDrive/bert/models/3
/content/drive/MyDrive/bert/models/4


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at foo and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
MODEL_PATH = os.path.join(BASE_PATH, 'models')
print(MODEL_PATH)

/content/drive/MyDrive/bert/models


## Load NLP training tasks

### CoLA: Sequence classification

In [ ]:
#@title Load CoLA data

def get_cola():
    COLA_TRAIN_FNAME = 'cola/in_domain_train.tsv' # Be sure to load this into environment
    COLA_TRAIN_PATH = os.path.join(DATA_PATH, COLA_TRAIN_FNAME)
    df = pd.read_csv(COLA_TRAIN_PATH, delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
    print(f"Data frame shape: {df.shape}")

    # sentences = ["[CLS] " + sentence + " [SEP]" for sentence in df.sentence.values]
    labels = df.label.values
    return sentences, labels, df

## Training run agenda

1. Instantiate a base model & head
2. Train on an NLP task according to the head type
3. Save the base model
4. Instantiate new head using saved base model as pretrain



In [ ]:
# Instantiate a model & head


## Load CoLA data

In [ ]:
#@title Load CoLA data

def get_cola():
    COLA_TRAIN_FNAME = 'cola/in_domain_train.tsv' # Be sure to load this into environment
    COLA_TRAIN_PATH = os.path.join(DATA_PATH, COLA_TRAIN_FNAME)
    df = pd.read_csv(COLA_TRAIN_PATH, delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
    print(f"Data frame shape: {df.shape}")

    sentences = ["[CLS] " + sentence + " [SEP]" for sentence in df.sentence.values]
    labels = df.label.values
    return sentences, labels, df

## COLA pseudocode

In [ ]:
# sentences, labels = get_cola()

# construct mask sequences?
    # get max sequence length from data
    # # Convert tokens to embedded vectors
# input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
# print(input_ids[0])

# # Pad the input tokens to uniform max length
# input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
#                           truncating="post", 
#                           padding='post')
# print(input_ids[0])
# # Create a binary mask over each sequence
attention_masks = [[float(elt > 0) for elt in seq] for seq in input_ids]

# Seems like the zero-ness of the padded elements would be a giveaway
X_train, X_test, y_train, y_test = train_test_split(input_ids, labels, 
                                                    random_state=42, test_size=0.1)
train_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids,
                                                 random_state = 42, test_size=0.1)

# Convert to torch

X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
train_masks = torch.tensor(train_masks)
test_masks = torch.tensor(test_masks)

# Create iterator for data loader

BATCH_SIZE = 32

train_data = TensorDataset(X_train, train_masks, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(X_test, test_masks, y_test)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
sentences, labels, df = get_cola()

Data frame shape: (8551, 4)


In [ ]:
# sentences = ["[CLS] " + sentence + " [SEP]" for sentence in df.sentence.values]
df.sentence.values


array([nan, nan, nan, ..., nan, nan, nan])

In [ ]:
#@ Instantiate a pretrained BERT tokenizer
from transformers import BertTokenizer

tokenizer_pt = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_text = [tokenizer_pt.tokenize(sentence) for sentence in sentences]
print(tokenized_text[0])


['[CLS]', 'our', 'friends', 'wo', 'n', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '[SEP]']


In [ ]:
tokenizer.encode(sentences[0]).tokens

['[',
 'CL',
 'S',
 ']',
 'Ġour',
 'Ġfriends',
 'Ġw',
 'o',
 'Ġn',
 "'t",
 'Ġbu',
 'y',
 'Ġthis',
 'Ġanalysis',
 'Ġ',
 ',',
 'Ġlet',
 'Ġalone',
 'Ġthe',
 'Ġnext',
 'Ġone',
 'Ġwe',
 'Ġpropose',
 'Ġ',
 '.',
 'Ġ[',
 'SE',
 'P',
 ']']